In [1]:
import numpy as np
import os
import pickle
from time import time
from datetime import datetime

In [2]:
import pulse2percept as p2p
import p2pspatial

2018-01-07 12:59:04,396 [pulse2percept] [INFO] Welcome to pulse2percept


In [3]:
subject = '12-005'
amplitude = 2.0
electrodes = None
random_state = 42
n_folds = 5

In [4]:
t_start = time()
now = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
filename = 'crossval-swarm_%s_%s.pickle' % (subject, now)
print(filename)

crossval-swarm_12-005_2018-01-07_12-59-04.pickle


In [5]:
rootfolder = os.path.join(os.environ['SECOND_SIGHT_DATA'], 'shape')
X, y = p2pspatial.load_data(rootfolder, subject=subject, electrodes=electrodes,
                            amplitude=amplitude, random_state=random_state,
                            single_stim=True, verbose=True)
print(X.shape, y.shape)
if len(X) == 0:
    raise ValueError('no data found')

Found 60 samples in C:\Users\mbeyeler\data\secondsight\shape\12-005\20091028\1.5xTh 20Hz 0.45ms 250ms
Found 60 samples in C:\Users\mbeyeler\data\secondsight\shape\12-005\20091028\5xTh 20Hz 0.45ms 250ms
Found 60 samples in C:\Users\mbeyeler\data\secondsight\shape\12-005\20100127\1.25xTh 20Hz 0.45ms 250ms set A
Found 60 samples in C:\Users\mbeyeler\data\secondsight\shape\12-005\20100127\1.5xTh 20Hz 0.45ms 250ms set A
Found 60 samples in C:\Users\mbeyeler\data\secondsight\shape\12-005\20100127\1.5xTh 20Hz 0.45ms 250ms set B
Found 60 samples in C:\Users\mbeyeler\data\secondsight\shape\12-005\20100127\2xTh 20Hz 0.45ms 250ms set A
Found 60 samples in C:\Users\mbeyeler\data\secondsight\shape\12-005\20100127\2xTh 20Hz 0.45ms 250ms set B
Found 60 samples in C:\Users\mbeyeler\data\secondsight\shape\12-005\20100203\2xTh 20Hz 0.45ms 250ms set A
Found 35 samples in C:\Users\mbeyeler\data\secondsight\shape\12-005\20100428\1.5xTh 20Hz 0.45ms 250ms set A
Found 40 samples in C:\Users\mbeyeler\data\seco

In [6]:
model_params = {'sampling': 200,
                'csmode': 'gaussian',
                'sensitivity_rule': 'decay',
                'thresh': 1.0}
regressor = p2pspatial.SpatialModelRegressor(**model_params)

In [7]:
search_params = {'decay_const': (0.001, 10),
                 'cswidth': (10, 1000),
                 'implant_x': (-1500, 1500),
                 'implant_y': (-1000, 1000),
                 'loc_od_x': (13, 17),
                 'loc_od_y': (0, 4),
                 'implant_rot': np.deg2rad((-75, -15))}
pso_options = {'max_iter': 100,
               'min_func': 1e-4}
pso = p2pspatial.model_selection.ParticleSwarmOptimizer(
    regressor, search_params, **pso_options
)

In [8]:
fit_params = {'scoring_weights': {'area': 0.001,
                                  'orientation': 100.0,
                                  'major_axis_length': 0.1,
                                  'minor_axis_length': 0.1}}
y_test, y_pred, best_params = p2pspatial.model_selection.crossval_predict(
    pso, X, y, fit_params=fit_params, n_folds=n_folds)

No constraints given.
implant (x, y): (-542.73, 589.13), rot: -0.817211
Set loc_od: 16.3127585809 2.47006067502 decay_const: 5.06276137187 sensitivity_rule: decay thresh: 1.0
Calculating effective current for electrodes: ['A01' 'A04' 'A06' 'A08' 'A10' 'B01' 'B03' 'B09' 'C04' 'C07' 'C08' 'C10'
 'D01' 'D04' 'D07' 'D08' 'D10' 'E01' 'E03' 'E07' 'E10' 'F02' 'F04' 'F06'
 'F09']
Done.


c:\users\mbeyeler\miniconda3\lib\site-packages\p2pspatial-0.1.dev0-py3.5.egg\p2pspatial\p2pspatial.py:414: RuntimeWarning: invalid value encountered in remainder


orientation isnan [        nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan  0.05034511
  0.34296308  0.26248386  0.8977652   0.03681596  0.15279726  1.01416285
  1.27627448  0.21519627  0.03973

c:\users\mbeyeler\miniconda3\lib\site-packages\p2pspatial-0.1.dev0-py3.5.egg\p2pspatial\p2pspatial.py:414: RuntimeWarning: invalid value encountered in remainder


orientation isnan [        nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan  2.15002979
  0.9433139   0.91463129  1.40918272  0.83374367  2.04970957  0.75695946
  1.8766253   0.45228203  2.34668

...

c:\users\mbeyeler\miniconda3\lib\site-packages\p2pspatial-0.1.dev0-py3.5.egg\p2pspatial\p2pspatial.py:414: RuntimeWarning: invalid value encountered in remainder


orientation isnan [             nan              nan              nan              nan
              nan              nan              nan              nan
              nan              nan              nan              nan
              nan              nan              nan              nan
              nan              nan              nan              nan
              nan              nan              nan              nan
              nan              nan              nan              nan
              nan              nan              nan              nan
              nan              nan              nan              nan
              nan              nan              nan              nan
              nan              nan              nan              nan
              nan              nan              nan              nan
              nan              nan              nan              nan
              nan              nan              nan              nan
              na

c:\users\mbeyeler\miniconda3\lib\site-packages\p2pspatial-0.1.dev0-py3.5.egg\p2pspatial\p2pspatial.py:414: RuntimeWarning: invalid value encountered in remainder


orientation isnan [        nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan  0.35752226
  0.01740509  0.60514606  0.88776849  0.09139941  0.36437113  0.8569266
  0.95071649  0.2929785   0.049736

KeyboardInterrupt: 

In [10]:
for k, v in search_params.items():
    setattr(regressor, k, v[0])

In [11]:
regressor.fit(X, y=y, **fit_params)

implant (x, y): (-1500.00, -1000.00), rot: -1.308997
Set loc_od: 13 0 decay_const: 0.001 sensitivity_rule: decay thresh: 1.0
Calculating effective current for electrodes: ['A01' 'A04' 'A06' 'A08' 'A10' 'B01' 'B03' 'B09' 'C04' 'C07' 'C08' 'C10'
 'D01' 'D04' 'D07' 'D08' 'D10' 'E01' 'E03' 'E07' 'E10' 'F02' 'F04' 'F06'
 'F09']
Done.


SpatialModelRegressor(csmode='gaussian', sampling=200,
           sensitivity_rule='decay', thresh=1.0, x_range=(-30, 30),
           y_range=(-20, 20))

In [12]:
regressor.score(X, y)

RMSE: 178.813691751


178.81369175079496

In [ ]:
print("Done in %.3fs" % (time() - t_start))

In [ ]:
specifics = {'subject': subject,
             'amplitude': amplitude,
             'electrodes': electrodes,
             'n_folds': n_folds,
             'regressor': regressor,
             'optimizer': pso,
             'model_params': model_params,
             'search_params': search_params,
             'fit_params': fit_params,
             'now': now,
             'random_state': random_state}
pickle.dump((y_test, y_pred, best_params, specifics), open(filename, 'wb'))
print('Dumped data to %s' % filename)